In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import litellm

response = litellm.completion(
    model="gpt-4o-mini-2024-07-18",
    messages=[{"role": "user", "content": "hi"}],
    num_retries=10,
)
response.choices[0].message.content

In [3]:
import base64
from openai import OpenAI
import os

import os
base_url = os.environ.get("OPENAI_BASE_URL")
api_key = os.environ.get("OPENAI_API_KEY", "https://openrouter.ai/api/v1")

client = OpenAI(
    api_key=api_key, base_url=base_url
)

# # Path to your image
# image_path = "/mnt/moonfs/wuyuhao-m2/wyh/test/test_openai.png"

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Getting the base64 string
# base64_image = encode_image(image_path)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "请分析这张图片"
                },
                # {
                #     "type": "image_url",
                #     "image_url": {
                #         "url": f"data:image/png;base64,{base64_image}"
                #     }
                # }
            ],
        }
    ],
    model="gpt-4o",
)
print(chat_completion.choices[0].message.content)

很抱歉，我无法查看或分析任何图片。如果你能描述一下图片的内容，我将很乐意帮助你分析或者提供相关信息。
